# May 14, 2025: 3d render canonical systems
rsns

In [1]:
import csv
import os
import sys
import numpy as np
import pandas as pd
import scipy as sp 
import dill as pickle 
from os.path import join as pjoin
from itertools import product
from tqdm import tqdm
from copy import deepcopy
from pathlib import Path
import subprocess
from scipy import sparse, stats
from multiprocessing import Pool
import glob
import random
import re

from sklearn.utils import resample

import arviz as az

import ants
import nibabel as nib

from itertools import product, combinations, chain
import multiprocessing as mp
from functools import partial
from multiprocessing import Process

# networks
import graph_tool.all as gt

# 3d rendering
from brainrender import Scene, actor

import pyvista as pv
from vedo import Mesh, settings
settings.default_backend = 'k3d'
from vedo import Volume, show
from skimage.measure import marching_cubes
from scipy.ndimage import label, binary_fill_holes, binary_closing

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cm import rainbow
from matplotlib.colors import Normalize

plt.rcParamsDefault['font.family'] = "sans-serif"
plt.rcParamsDefault['font.sans-serif'] = "Arial"
plt.rcParams['font.size'] = 14
plt.rcParams["errorbar.capsize"] = 0.5

import cmasher as cmr  # CITE ITS PAPER IN YOUR MANUSCRIPT
import colorcet as cc

# ignore user warnings
import warnings
warnings.filterwarnings("ignore") #, category=UserWarning)

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

def set_seed(args):
    gt.seed_rng(args.SEED)
    np.random.seed(args.SEED)

set_seed(args)

In [3]:
args.source ='allen' #'spatial' #'allen'
args.space = 'ccfv2' #'yale' #'ccfv2'
args.resolution = 200 #200 #200

TEMP_DESC = (
    f'source-{args.source}'
    f'_space-{args.space}'
    f'_res-{args.resolution}'
)
TEMP_DESC

'source-allen_space-ccfv2_res-200'

In [4]:
args.GRAPH_DEF = f'constructed'
args.GRAPH_METHOD = f'pearson'
args.THRESHOLD = f'signed'
args.EDGE_DEF = f'binary'
args.EDGE_DENSITY = 20
args.LAYER_DEF = f'individual'
args.DATA_UNIT = f'sub'

BASE_path = f'{os.environ["HOME"]}/new_mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
RSNS_path = f'{BASE_path}/canonical_systems/joanes/space-{args.space}/rsns/both'

In [5]:
template_file = f'{PARCELS_path}/{TEMP_DESC}_desc-template.nii.gz'
template_img = nib.load(template_file)

In [6]:
rsn_files = sorted(glob.glob(f'{RSNS_path}/*desc-*.nii.gz', recursive=True))
rsn_files


[
    '/home/govindas/new_mouse_dataset/canonical_systems/joanes/space-ccfv2/rsns/both/hemi-B_desc-00.nii.gz',
    '/home/govindas/new_mouse_dataset/canonical_systems/joanes/space-ccfv2/rsns/both/hemi-B_desc-01.nii.gz',
    '/home/govindas/new_mouse_dataset/canonical_systems/joanes/space-ccfv2/rsns/both/hemi-B_desc-02.nii.gz',
    '/home/govindas/new_mouse_dataset/canonical_systems/joanes/space-ccfv2/rsns/both/hemi-B_desc-03.nii.gz',
    '/home/govindas/new_mouse_dataset/canonical_systems/joanes/space-ccfv2/rsns/both/hemi-B_desc-04.nii.gz'
]

In [7]:
rsn_imgs = [
    nib.load(rsn_file)
    for rsn_file in rsn_files
]

In [8]:
def taubin_smooth_vedo_mesh(mesh: Mesh, n_iter=50, pass_band=0.5) -> Mesh:
    vtk_poly = mesh.dataset
    pv_mesh = pv.wrap(vtk_poly)
    smoothed = pv_mesh.smooth_taubin(n_iter=n_iter, pass_band=pass_band)
    return Mesh(smoothed)

def create_mesh_from_volume(
        volume: np.ndarray,
        threshold: float = 75,
        spacing: tuple = (200, 200, 200),
        origin: tuple = (0, 0, 0),
        smoothing: dict = {'n_iter': 50, 'pass_band': 0.5},
        color: str = 'cornflowerblue',
        alpha: float = 1.0,
) -> Mesh:
    threshold = np.percentile(volume, threshold)
    verts, faces, *_ = marching_cubes(volume.astype(np.uint8), level=threshold)

    verts = np.asarray(spacing) * verts + np.asarray(origin)
    faces = faces.astype(np.uint32)
    faces_vtk = np.c_[np.full(len(faces), 3), faces].flatten()

    mesh = Mesh([verts, faces_vtk])
    mesh = taubin_smooth_vedo_mesh(mesh, **smoothing)
    mesh.color(color).alpha(alpha)
    return mesh

In [9]:
def create_brain_surface(
    template_img: nib.Nifti1Image,
    threshold: float = 75,
    resolution: int = 200,
    smoothing: dict = {'n_iter': 100, 'pass_band': 0.5},
    color: str = '#cccccc',
    alpha: float = 0.1,
    l: int = 2,
) -> Mesh:
    # Load and reorient
    volume = template_img.get_fdata()
    volume = np.transpose(volume, (2, 1, 0))[::-1, :, :]
    
    # Threshold
    tval = np.percentile(volume, threshold)
    mask = (volume >= tval).astype(np.uint8)
    mask = binary_fill_holes(mask)
    mask = binary_closing(mask, structure=np.ones((l, l, l)))
    
    # Marching cubes on thresholded mask
    verts, faces, *_ = marching_cubes(mask, level=0.5)
    spacing = np.array([resolution] * 3)
    origin = np.array([0] * 3)
    verts = spacing * verts + origin
    faces = faces.astype(np.uint32)
    faces_vtk = np.c_[np.full(len(faces), 3), faces].flatten()

    # Create mesh
    mesh = Mesh([verts, faces_vtk])

    # Split into connected pieces, keep largest
    mesh_pieces = mesh.split(maxdepth=1, sort_by_area=True)
    largest = max(mesh_pieces, key=lambda m: m.npoints)

    # Smooth and color
    largest = taubin_smooth_vedo_mesh(largest, **smoothing)
    largest.color(color).alpha(alpha)

    return largest
    return template_mesh

In [10]:
args.threshold = 75

template_mesh = create_brain_surface(
    template_img=template_img,
    threshold=args.threshold,
    resolution=args.resolution,
    smoothing=dict(n_iter=100, pass_band=0.2),
)

In [11]:
rsn_meshes = []
for idx, rsn_img in enumerate(rsn_imgs):
    volume = (rsn_img.get_fdata() == 1.0).astype(np.uint8)
    volume = np.transpose(volume, (2, 1, 0))[::-1, :, :]
    rsn_mesh = create_mesh_from_volume(
        volume=volume,
        threshold=75,
        color='#d55e00',
    )
    rsn_meshes += [rsn_mesh]

In [12]:
for rsn_file, rsn_mesh in zip(rsn_files, rsn_meshes):
    scene = Scene(atlas_name='allen_mouse_100um')
    scene.root.alpha(0.0)
    scene.add(actor.Actor(template_mesh, br_class='Volume'))
    scene.add(actor.Actor(rsn_mesh, br_class='Volume'))
    out_file = rsn_file.replace('.nii.gz', '.html')
    scene.export(f'{out_file}')

Cannot run function _get_inset in a jupyter notebook Try setting the correct backend before creating your scene:

from vedo import embedWindow                                                                                       
embedWindow(None)                                                                                                  

Your scene is ready for rendering, use:

from vedo import show                                                                                              
vedo.show(*scene.renderables)                                                                                      

The brainrender scene has been exported for web. The results are saved at 
/home/govindas/new_mouse_dataset/canonical_systems/joanes/space-ccfv2/rsns/both/hemi-B_desc-00.html

Cannot run function _get_inset in a jupyter notebook Try setting the correct backend before creating your scene:

from vedo import embedWindow                                                                                       
embedWindow(None)                                                                                                  

Your scene is ready for rendering, use:

from vedo import show                                                                                              
vedo.show(*scene.renderables)                                                                                      

...

Your scene is ready for rendering, use:

from vedo import show                                                                                              
vedo.show(*scene.renderables)                                                                                      

The brainrender scene has been exported for web. The results are saved at 
/home/govindas/new_mouse_dataset/canonical_systems/joanes/space-ccfv2/rsns/both/hemi-B_desc-03.html

Cannot run function _get_inset in a jupyter notebook Try setting the correct backend before creating your scene:

from vedo import embedWindow                                                                                       
embedWindow(None)                                                                                                  

Your scene is ready for rendering, use:

from vedo import show                                                                                              
vedo.show(*scene.renderables)                                                                                      

The brainrender scene has been exported for web. The results are saved at 
/home/govindas/new_mouse_dataset/canonical_systems/joanes/space-ccfv2/rsns/both/hemi-B_desc-04.html